In [ ]:
!pip install -e .

In [ ]:
!pip install --upgrade torch

In [1]:
import torch
import torch.onnx

In [2]:
torch.__version__


'1.11.0+cu102'

In [3]:
from uberduck_ml_dev.models.tacotron2 import Tacotron2, Tacotron2ForwardIsInfer, DEFAULTS
from uberduck_ml_dev.data_loader import prepare_input_sequence
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
from uberduck_ml_dev.models.common import griffin_lim, MelSTFT

In [4]:
stft = MelSTFT()

In [5]:
vals = DEFAULTS.values()
vals.update(cudnn_enabled=False)
hp = HParams(**vals)
model = Tacotron2ForwardIsInfer(hp)

Not using any style tokens


In [6]:
sd = torch.load("model_store/tacotron2-trebek-arpa-740-01-02-21.pt", map_location="cpu")

In [7]:
model.load_state_dict(sd)

<All keys matched successfully>

In [7]:
input_sequence = prepare_input_sequence(["The quick brown fox jumped over the lazy dog and then pooped in my mouth."], cpu_run=True, arpabet=True)
inputs, input_lengths = input_sequence

In [8]:
from typing import List

input_: List[torch.Tensor] = [inputs, input_lengths, torch.tensor([0]), torch.tensor([])]

In [10]:
output = model.inference(input_)


In [9]:
model.eval()
#model.forward = model.inference
#model.encoder.forward = model.encoder.inference
scripted_module = torch.jit.script(model)

In [12]:
out = scripted_module(input_)
for o in out:
    print(o.shape)

out_postnet = out[1]
audio = stft.griffin_lim(out_postnet[0])
from IPython.display import Audio
Audio(audio, rate=22050)

torch.Size([1, 80, 399])
torch.Size([1, 399])
torch.Size([1, 399, 63])
torch.Size([1])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [1]:

torch.onnx.export(
    scripted_module,
    (input_,),
    "test.onnx", 
    verbose=True, 
    opset_version=15,
    dynamic_axes=dict(
        mel_out={0: "batch_size", 2: "output_length"}, 
        mel_postnet={0: "batch_size", 2: "output_length"}, 
        gate={0: "batch_size", 1: "output_length"}, 
        attn={0: "batch_size", 1: "output_length", 2: "sequence_length"},
        input={0: "batch_size", 1: "sequence_length"},                 
    ),
    input_names=["input", "input_lengths", "speaker_id", "gst"],
    output_names=["mel_out", "mel_postnet", "gate", "attn", "output_frames"],
)

NameError: name 'torch' is not defined

In [61]:
#model.eval()
#model.forward = model.inference
torch.onnx.export(scripted_module, (input_,), "test.onnx", verbose=True, opset_version=11,
                  dynamic_axes=dict(mel_out={0: "batch_si2], mel_postnet=[2], gate=[1], attn=[1]),
                  input_names=["input", "input_lengths", "speaker_id", "gst"],
                  output_names=["mel_out", "mel_postnet", "gate", "attn", "output_frames"],
#     output_names=[
#         ["mel_out_bsize", "mel_out_channels", "mel_out_frames"],
#         ["mel_postnet_bsize", "mel_postnet_channels", "mel_postnet_frames"],
#         ["gate_bsize", "gate_frames"],
#         ["attn_bsize", "attn_frames", "attn_input_size"],
#         ["output_frames"]
#     ],
)

/usr/local/lib/python3.8/site-packages/torch/onnx/utils.py:1329: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input mel_out
  warnings.warn("No names were found for specified dynamic axes of provided input."
/usr/local/lib/python3.8/site-packages/torch/onnx/utils.py:1329: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input mel_postnet
  warnings.warn("No names were found for specified dynamic axes of provided input."
/usr/local/lib/python3.8/site-packages/torch/onnx/utils.py:1329: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input gate
  warnings.warn("No names were found for specified dynamic axes of provided input."
/usr/local/lib/python3.8/site-packages/torch/onnx/utils.py:1329: U

RuntimeError: Unsupported: ONNX export of transpose for tensor of unknown rank.

In [ ]:
DEFAULTS.cudnn_enabled

In [ ]:
?model.inference

In [ ]:
!pip install torchvision

In [ ]:
import torchvision.models as models

In [ ]:
import torch

In [ ]:
model = models.resnet50(pretrained=True)

In [ ]:
model.eval()

In [ ]:
dummy_input = torch.randn(1, 3, 224, 224)

In [ ]:
input_names = [ "actual_input" ]
output_names = [ "output" ]

In [ ]:
torch.onnx.export(model, 
                  dummy_input,
                  "resnet50.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names,
                  export_params=True,
                  )

In [ ]:
!ls -lash resnet50.onnx

In [ ]:
torch.__version__

In [ ]:
!pip install --upgrade onnx

In [ ]:
!pip install --upgrade torch

In [ ]:
torch.__version__

In [ ]:
x = 0
for i in range(100):
    if (torch.rand(1) < 0.5):
        x += 1
print(x / 100)

In [ ]:
x = torch.rand(1) < 0.5
print(x)
x.all()

In [30]:
model.gst_lin

In [19]:
scripted_module.save("scripted-tacotron2.pt")